In [93]:
DATA_PATH = "/home/giovanni/pod/thesis/code/scripts-sources/W3Pi-selection-algorithm/data/"

import uproot
import awkward as ak
import numpy as np
import pandas as pd

with uproot.open(DATA_PATH + "l1Nano_WTo3Pion_PU200.root") as f:
    tree = f.get("Events")

    # assign the columns (branches) for the tree FOR EACH EVENT to branches
    branches = tree.arrays()

    
    event_idx = 6
    event = branches[event_idx]
    
    GenW_mass = event["GenW_mass"]
    nPuppi = event["nPuppi"]

    columns = ["Puppi_pdgId", "Puppi_GenPiIdx"]
    df_data = {col: event[col] for col in columns}
    df = pd.DataFrame(df_data)
    print(df)
    

    Puppi_pdgId  Puppi_GenPiIdx
0           211              -1
1           211              -1
2           211              -1
3            11              -1
4           211              -1
5           211              -1
6           211              -1
7          -211              -1
8          -211              -1
9           211              -1
10         -211              -1
11          211              -1
12         -211              -1
13          211              -1
14          130              -1
15          211              -1
16         -211              -1
17         -211               0
18         -211               1
19          130              -1
20           22              -1
21           22              -1
22         -211              -1
23         -211              -1
24          211              -1
25          211               2
26          211              -1
27          211              -1
28          211              -1
29          130              -1
30      